## 0. 데이터셋 다운로드

In [1]:
!chmod +x download.sh
!./download.sh
!unzip -q CelebA_128crop_FD.zip?dl=0 -d ./data/

--2017-12-16 11:20:02--  https://www.dropbox.com/s/e0ig4nf1v94hyj8/CelebA_128crop_FD.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.1, 2620:100:6030:1::a27d:5001
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dl.dropboxusercontent.com/content_link/4sqb0hC1xkohlpZvZwhXzZUEy5aPZCesgzlKwcSdC552DShNqr8B5aJeVWvVqBUO/file [following]
--2017-12-16 11:20:03--  https://dl.dropboxusercontent.com/content_link/4sqb0hC1xkohlpZvZwhXzZUEy5aPZCesgzlKwcSdC552DShNqr8B5aJeVWvVqBUO/file
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.80.6, 2620:100:6030:6::a27d:5006
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.80.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1090119409 (1.0G) [application/zip]
Saving to: ‘./CelebA_128crop_FD.zip?dl=0’

100%[====================================>] 1,090,119,409 36.4MB/

## 1. 모듈/라이브러리 임포트

In [2]:
import torch, os
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import save_image
from torch.autograd import Variable
from torch.utils.data import DataLoader

## 2. 하이퍼 파라메터

In [3]:
lr = 0.0002
max_epoch = 20
batch_size = 32
z_dim = 100
image_size = 64
g_conv_dim = 64
d_conv_dim = 64
log_step = 100
sample_step = 500
sample_num = 32
image_path = './data/CelebA/'
sample_path = './output/' 
if not os.path.exists(sample_path) : os.makedirs(sample_path)

## 3. 데이터 준비

In [4]:
transform = transforms.Compose([
                transforms.Resize(image_size),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

dataset = ImageFolder(image_path, transform)
data_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=8,
                          drop_last=True)

## 4. 모델 정의
<br><br>
![iImage](dcgan.png)

In [5]:
def deconv(c_in, c_out, k_size, stride=2, pad=1, bn=True):
    """Custom deconvolutional layer for simplicity."""
    layers = []
    layers.append(nn.ConvTranspose2d(c_in, c_out, k_size, stride, pad))
    if bn:
        layers.append(nn.BatchNorm2d(c_out))
    return nn.Sequential(*layers)


class Generator(nn.Module):
    """Generator containing 7 deconvolutional layers."""
    def __init__(self, z_dim=256, image_size=128, conv_dim=64):
        super(Generator, self).__init__()
        self.fc = deconv(z_dim, conv_dim*8, int(image_size/16), 1, 0, bn=False)
        self.deconv1 = deconv(conv_dim*8, conv_dim*4, 4)
        self.deconv2 = deconv(conv_dim*4, conv_dim*2, 4)
        self.deconv3 = deconv(conv_dim*2, conv_dim, 4)
        self.deconv4 = deconv(conv_dim, 3, 4, bn=False)
        
    def forward(self, z):
        z = z.view(z.size(0), z.size(1), 1, 1)      # If image_size is 64, output shape is as below.
        out = self.fc(z)                            # (?, 512, 4, 4)
        out = F.leaky_relu(self.deconv1(out), 0.05)  # (?, 256, 8, 8)
        out = F.leaky_relu(self.deconv2(out), 0.05)  # (?, 128, 16, 16)
        out = F.leaky_relu(self.deconv3(out), 0.05)  # (?, 64, 32, 32)
        out = F.tanh(self.deconv4(out))             # (?, 3, 64, 64)
        return out
    
    
def conv(c_in, c_out, k_size, stride=2, pad=1, bn=True):
    """Custom convolutional layer for simplicity."""
    layers = []
    layers.append(nn.Conv2d(c_in, c_out, k_size, stride, pad))
    if bn:
        layers.append(nn.BatchNorm2d(c_out))
    return nn.Sequential(*layers)


class Discriminator(nn.Module):
    """Discriminator containing 4 convolutional layers."""
    def __init__(self, image_size=128, conv_dim=64):
        super(Discriminator, self).__init__()
        self.conv1 = conv(3, conv_dim, 4, bn=False)
        self.conv2 = conv(conv_dim, conv_dim*2, 4)
        self.conv3 = conv(conv_dim*2, conv_dim*4, 4)
        self.conv4 = conv(conv_dim*4, conv_dim*8, 4)
        self.fc = conv(conv_dim*8, 1, int(image_size/16), 1, 0, False)
        
    def forward(self, x):                         # If image_size is 64, output shape is as below.
        out = F.leaky_relu(self.conv1(x), 0.05)    # (?, 64, 32, 32)
        out = F.leaky_relu(self.conv2(out), 0.05)  # (?, 128, 16, 16)
        out = F.leaky_relu(self.conv3(out), 0.05)  # (?, 256, 8, 8)
        out = F.leaky_relu(self.conv4(out), 0.05)  # (?, 512, 4, 4)
        out = F.sigmoid(self.fc(out)).squeeze()
#         out = self.fc(out).squeeze() # Least Square
        return out
    
D = Discriminator(image_size)
G = Generator(z_dim,image_size,g_conv_dim)

D.cuda()
G.cuda()

Generator(
  (fc): Sequential(
    (0): ConvTranspose2d (100, 512, kernel_size=(4, 4), stride=(1, 1))
  )
  (deconv1): Sequential(
    (0): ConvTranspose2d (512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
  )
  (deconv2): Sequential(
    (0): ConvTranspose2d (256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
  )
  (deconv3): Sequential(
    (0): ConvTranspose2d (128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  )
  (deconv4): Sequential(
    (0): ConvTranspose2d (64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  )
)

## 5. Loss/Optimizer

In [6]:
# Binary cross entropy loss and optimizer
criterion = nn.BCELoss().cuda()
d_optimizer = torch.optim.Adam(D.parameters(), lr=lr, betas=(0.5,0.999))
g_optimizer = torch.optim.Adam(G.parameters(), lr=lr, betas=(0.5,0.999))

## 6. 기타 함수

In [7]:
# denormalization : [-1,1] -> [0,1]
# normalization : [0,1] -> [-1,1]
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

## 7. 트레이닝

In [8]:
# Start training
total_batch = len(data_loader.dataset)//batch_size
fixed_z = Variable(torch.randn(sample_num, z_dim)).cuda()
for epoch in range(max_epoch):
    for i, (images, _) in enumerate(data_loader):
        # Build mini-batch dataset
        images = Variable(images).cuda()
        # Create the labels which are later used as input for the BCE loss
        real_labels = Variable(torch.ones(batch_size)).cuda()
        fake_labels = Variable(torch.zeros(batch_size)).cuda()

        #============= Train the discriminator =============#
        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels) # BCE
#         d_loss_real = torch.mean((outputs - 1) ** 2) # Least Square
        
        real_score = outputs

        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = Variable(torch.randn(batch_size, z_dim)).cuda()
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels) # BCE
#         d_loss_fake = torch.mean(outputs ** 2) # Least Square
        fake_score = outputs

        # Backprop + Optimize
        d_loss = d_loss_real + d_loss_fake
        D.zero_grad()
        d_loss.backward()
        d_optimizer.step()

        #=============== Train the generator ===============#
        # Compute loss with fake images
        z = Variable(torch.randn(batch_size, z_dim)).cuda()
        fake_images = G(z)
        outputs = D(fake_images)

        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels) # BCE
#        g_loss = torch.mean((outputs - 1) ** 2) # Least Square

        # Backprop + Optimize
        D.zero_grad()
        G.zero_grad()
        g_loss.backward()
        g_optimizer.step()

        if (i+1) % log_step == 0:
            print('Epoch [%d/%d], Step[%d/%d], d_loss: %.4f, '
                  'g_loss: %.4f, D(x): %.2f, D(G(z)): %.2f'
                  %(epoch, max_epoch, i+1, total_batch, d_loss.data[0], g_loss.data[0],
                    real_score.data.mean(), fake_score.data.mean()))
            
        if (i+1) % sample_step == 0:
            fake_images = G(fixed_z)
            torchvision.utils.save_image(denorm(fake_images.data), 
                                         os.path.join(sample_path,'fake_samples-%d-%d.png' %(epoch+1, i+1)),
                                         nrow=8)

# Save the trained parameters
torch.save(G.state_dict(), './generator.pkl')
torch.save(D.state_dict(), './discriminator.pkl')

Epoch [0/20], Step[100/4882], d_loss: 0.7047, g_loss: 4.3068, D(x): 0.81, D(G(z)): 0.32
Epoch [0/20], Step[200/4882], d_loss: 0.8299, g_loss: 1.4400, D(x): 0.57, D(G(z)): 0.17
Epoch [0/20], Step[300/4882], d_loss: 1.0945, g_loss: 1.9409, D(x): 0.79, D(G(z)): 0.53
Epoch [0/20], Step[400/4882], d_loss: 1.1137, g_loss: 1.2201, D(x): 0.77, D(G(z)): 0.53
Epoch [0/20], Step[500/4882], d_loss: 0.7870, g_loss: 0.9941, D(x): 0.61, D(G(z)): 0.20
Epoch [0/20], Step[600/4882], d_loss: 1.1292, g_loss: 0.7774, D(x): 0.46, D(G(z)): 0.17
Epoch [0/20], Step[700/4882], d_loss: 0.9394, g_loss: 1.0819, D(x): 0.62, D(G(z)): 0.32
Epoch [0/20], Step[800/4882], d_loss: 0.9757, g_loss: 2.0048, D(x): 0.71, D(G(z)): 0.43
Epoch [0/20], Step[900/4882], d_loss: 0.8618, g_loss: 2.2981, D(x): 0.79, D(G(z)): 0.42
Epoch [0/20], Step[1000/4882], d_loss: 0.7863, g_loss: 1.2797, D(x): 0.57, D(G(z)): 0.14
Epoch [0/20], Step[1100/4882], d_loss: 0.8305, g_loss: 2.2016, D(x): 0.75, D(G(z)): 0.37
Epoch [0/20], Step[1200/4882],

Epoch [1/20], Step[4600/4882], d_loss: 1.1459, g_loss: 2.3886, D(x): 0.78, D(G(z)): 0.40
Epoch [1/20], Step[4700/4882], d_loss: 0.4552, g_loss: 2.5300, D(x): 0.81, D(G(z)): 0.13
Epoch [1/20], Step[4800/4882], d_loss: 0.1744, g_loss: 2.3465, D(x): 0.92, D(G(z)): 0.07
Epoch [2/20], Step[100/4882], d_loss: 0.2454, g_loss: 2.6487, D(x): 0.90, D(G(z)): 0.10
Epoch [2/20], Step[200/4882], d_loss: 0.5697, g_loss: 2.5762, D(x): 0.68, D(G(z)): 0.06
Epoch [2/20], Step[300/4882], d_loss: 0.3551, g_loss: 3.6821, D(x): 0.88, D(G(z)): 0.17
Epoch [2/20], Step[400/4882], d_loss: 0.2271, g_loss: 1.2170, D(x): 0.88, D(G(z)): 0.08
Epoch [2/20], Step[500/4882], d_loss: 0.2484, g_loss: 3.3811, D(x): 0.97, D(G(z)): 0.15
Epoch [2/20], Step[600/4882], d_loss: 0.2165, g_loss: 3.3955, D(x): 0.98, D(G(z)): 0.15
Epoch [2/20], Step[700/4882], d_loss: 0.2145, g_loss: 2.8339, D(x): 0.97, D(G(z)): 0.16
Epoch [2/20], Step[800/4882], d_loss: 0.2208, g_loss: 2.3976, D(x): 0.99, D(G(z)): 0.17
Epoch [2/20], Step[900/4882],

Epoch [3/20], Step[4300/4882], d_loss: 0.0837, g_loss: 4.0848, D(x): 0.99, D(G(z)): 0.07
Epoch [3/20], Step[4400/4882], d_loss: 0.1778, g_loss: 4.0852, D(x): 0.92, D(G(z)): 0.08
Epoch [3/20], Step[4500/4882], d_loss: 0.0823, g_loss: 5.9790, D(x): 0.95, D(G(z)): 0.03
Epoch [3/20], Step[4600/4882], d_loss: 0.3231, g_loss: 4.9804, D(x): 0.89, D(G(z)): 0.14
Epoch [3/20], Step[4700/4882], d_loss: 0.2552, g_loss: 4.5107, D(x): 0.96, D(G(z)): 0.17
Epoch [3/20], Step[4800/4882], d_loss: 0.1602, g_loss: 4.0725, D(x): 0.99, D(G(z)): 0.13
Epoch [4/20], Step[100/4882], d_loss: 0.3295, g_loss: 5.2531, D(x): 0.78, D(G(z)): 0.01
Epoch [4/20], Step[200/4882], d_loss: 0.6108, g_loss: 5.4664, D(x): 1.00, D(G(z)): 0.38
Epoch [4/20], Step[300/4882], d_loss: 1.6823, g_loss: 9.7690, D(x): 1.00, D(G(z)): 0.64
Epoch [4/20], Step[400/4882], d_loss: 0.0560, g_loss: 5.0813, D(x): 0.95, D(G(z)): 0.00
Epoch [4/20], Step[500/4882], d_loss: 0.1183, g_loss: 5.0815, D(x): 0.94, D(G(z)): 0.04
Epoch [4/20], Step[600/488

Epoch [5/20], Step[4000/4882], d_loss: 0.0416, g_loss: 2.8021, D(x): 0.98, D(G(z)): 0.02
Epoch [5/20], Step[4100/4882], d_loss: 0.1116, g_loss: 6.4556, D(x): 0.97, D(G(z)): 0.05
Epoch [5/20], Step[4200/4882], d_loss: 0.0783, g_loss: 4.1986, D(x): 0.99, D(G(z)): 0.05
Epoch [5/20], Step[4300/4882], d_loss: 0.6884, g_loss: 6.1710, D(x): 0.63, D(G(z)): 0.00
Epoch [5/20], Step[4400/4882], d_loss: 0.0328, g_loss: 5.0465, D(x): 0.99, D(G(z)): 0.02
Epoch [5/20], Step[4500/4882], d_loss: 0.0764, g_loss: 7.0635, D(x): 0.97, D(G(z)): 0.04
Epoch [5/20], Step[4600/4882], d_loss: 0.1249, g_loss: 4.8512, D(x): 0.92, D(G(z)): 0.03
Epoch [5/20], Step[4700/4882], d_loss: 0.0250, g_loss: 4.9119, D(x): 0.99, D(G(z)): 0.02
Epoch [5/20], Step[4800/4882], d_loss: 0.1753, g_loss: 2.4560, D(x): 0.88, D(G(z)): 0.02
Epoch [6/20], Step[100/4882], d_loss: 0.1276, g_loss: 5.2488, D(x): 0.93, D(G(z)): 0.04
Epoch [6/20], Step[200/4882], d_loss: 0.0642, g_loss: 4.2037, D(x): 0.98, D(G(z)): 0.04
Epoch [6/20], Step[300/

Epoch [7/20], Step[3700/4882], d_loss: 0.1636, g_loss: 7.0951, D(x): 0.99, D(G(z)): 0.09
Epoch [7/20], Step[3800/4882], d_loss: 0.0675, g_loss: 5.4875, D(x): 0.95, D(G(z)): 0.01
Epoch [7/20], Step[3900/4882], d_loss: 0.0344, g_loss: 4.7860, D(x): 1.00, D(G(z)): 0.03
Epoch [7/20], Step[4000/4882], d_loss: 0.1265, g_loss: 4.7482, D(x): 0.93, D(G(z)): 0.01
Epoch [7/20], Step[4100/4882], d_loss: 0.0337, g_loss: 5.9268, D(x): 0.99, D(G(z)): 0.02
Epoch [7/20], Step[4200/4882], d_loss: 0.1548, g_loss: 5.6558, D(x): 0.90, D(G(z)): 0.00
Epoch [7/20], Step[4300/4882], d_loss: 0.1321, g_loss: 6.3271, D(x): 0.95, D(G(z)): 0.07
Epoch [7/20], Step[4400/4882], d_loss: 0.0494, g_loss: 4.8084, D(x): 0.99, D(G(z)): 0.03
Epoch [7/20], Step[4500/4882], d_loss: 0.5198, g_loss: 3.6028, D(x): 0.71, D(G(z)): 0.00
Epoch [7/20], Step[4600/4882], d_loss: 0.0532, g_loss: 6.7008, D(x): 0.96, D(G(z)): 0.01
Epoch [7/20], Step[4700/4882], d_loss: 0.0456, g_loss: 5.4529, D(x): 0.98, D(G(z)): 0.02
Epoch [7/20], Step[48

...

Epoch [11/20], Step[3000/4882], d_loss: 0.2449, g_loss: 4.5874, D(x): 0.84, D(G(z)): 0.00
Epoch [11/20], Step[3100/4882], d_loss: 0.0546, g_loss: 5.5035, D(x): 0.96, D(G(z)): 0.01
Epoch [11/20], Step[3200/4882], d_loss: 0.0543, g_loss: 6.6084, D(x): 1.00, D(G(z)): 0.04
Epoch [11/20], Step[3300/4882], d_loss: 0.2587, g_loss: 3.6148, D(x): 0.84, D(G(z)): 0.01
Epoch [11/20], Step[3400/4882], d_loss: 0.0040, g_loss: 5.9052, D(x): 1.00, D(G(z)): 0.00
Epoch [11/20], Step[3500/4882], d_loss: 0.0486, g_loss: 5.4351, D(x): 1.00, D(G(z)): 0.04
Epoch [11/20], Step[3600/4882], d_loss: 0.0154, g_loss: 10.5983, D(x): 0.99, D(G(z)): 0.00
Epoch [11/20], Step[3700/4882], d_loss: 0.0950, g_loss: 3.1767, D(x): 0.93, D(G(z)): 0.01
Epoch [11/20], Step[3800/4882], d_loss: 0.0151, g_loss: 5.5395, D(x): 1.00, D(G(z)): 0.01
Epoch [11/20], Step[3900/4882], d_loss: 0.0392, g_loss: 4.1466, D(x): 0.98, D(G(z)): 0.02
Epoch [11/20], Step[4000/4882], d_loss: 0.0798, g_loss: 5.2610, D(x): 1.00, D(G(z)): 0.06
Epoch [11

Epoch [13/20], Step[2600/4882], d_loss: 0.0146, g_loss: 6.5992, D(x): 0.99, D(G(z)): 0.00
Epoch [13/20], Step[2700/4882], d_loss: 0.0146, g_loss: 7.5293, D(x): 0.99, D(G(z)): 0.01
Epoch [13/20], Step[2800/4882], d_loss: 0.0686, g_loss: 8.3346, D(x): 1.00, D(G(z)): 0.06
Epoch [13/20], Step[2900/4882], d_loss: 0.0240, g_loss: 7.4112, D(x): 1.00, D(G(z)): 0.02
Epoch [13/20], Step[3000/4882], d_loss: 0.0080, g_loss: 8.0823, D(x): 0.99, D(G(z)): 0.00
Epoch [13/20], Step[3100/4882], d_loss: 0.0335, g_loss: 4.1636, D(x): 0.99, D(G(z)): 0.02
Epoch [13/20], Step[3200/4882], d_loss: 0.0253, g_loss: 6.8653, D(x): 0.99, D(G(z)): 0.01
Epoch [13/20], Step[3300/4882], d_loss: 0.0120, g_loss: 8.0454, D(x): 1.00, D(G(z)): 0.01
Epoch [13/20], Step[3400/4882], d_loss: 0.0457, g_loss: 8.7792, D(x): 0.99, D(G(z)): 0.03
Epoch [13/20], Step[3500/4882], d_loss: 0.2417, g_loss: 5.3433, D(x): 0.84, D(G(z)): 0.00
Epoch [13/20], Step[3600/4882], d_loss: 0.0229, g_loss: 6.6339, D(x): 1.00, D(G(z)): 0.02
Epoch [13/

Epoch [15/20], Step[2200/4882], d_loss: 0.5939, g_loss: 8.3805, D(x): 0.66, D(G(z)): 0.00
Epoch [15/20], Step[2300/4882], d_loss: 0.0126, g_loss: 9.4274, D(x): 0.99, D(G(z)): 0.01
Epoch [15/20], Step[2400/4882], d_loss: 0.0214, g_loss: 8.6913, D(x): 0.99, D(G(z)): 0.01
Epoch [15/20], Step[2500/4882], d_loss: 0.0070, g_loss: 8.0010, D(x): 1.00, D(G(z)): 0.00
Epoch [15/20], Step[2600/4882], d_loss: 0.0033, g_loss: 8.6551, D(x): 1.00, D(G(z)): 0.00
Epoch [15/20], Step[2700/4882], d_loss: 0.0159, g_loss: 6.1377, D(x): 1.00, D(G(z)): 0.01
Epoch [15/20], Step[2800/4882], d_loss: 0.0022, g_loss: 8.0098, D(x): 1.00, D(G(z)): 0.00
Epoch [15/20], Step[2900/4882], d_loss: 0.0285, g_loss: 6.3860, D(x): 0.98, D(G(z)): 0.00
Epoch [15/20], Step[3000/4882], d_loss: 0.0069, g_loss: 8.0401, D(x): 1.00, D(G(z)): 0.00
Epoch [15/20], Step[3100/4882], d_loss: 0.0097, g_loss: 8.4604, D(x): 1.00, D(G(z)): 0.01
Epoch [15/20], Step[3200/4882], d_loss: 0.1671, g_loss: 6.6335, D(x): 0.91, D(G(z)): 0.02
Epoch [15/

Epoch [17/20], Step[1800/4882], d_loss: 0.1510, g_loss: 5.5939, D(x): 0.91, D(G(z)): 0.04
Epoch [17/20], Step[1900/4882], d_loss: 0.0657, g_loss: 5.0861, D(x): 0.95, D(G(z)): 0.00
Epoch [17/20], Step[2000/4882], d_loss: 0.0069, g_loss: 9.6318, D(x): 0.99, D(G(z)): 0.00
Epoch [17/20], Step[2100/4882], d_loss: 0.0700, g_loss: 7.9737, D(x): 1.00, D(G(z)): 0.06
Epoch [17/20], Step[2200/4882], d_loss: 0.0064, g_loss: 8.1972, D(x): 1.00, D(G(z)): 0.00
Epoch [17/20], Step[2300/4882], d_loss: 0.1958, g_loss: 6.8493, D(x): 1.00, D(G(z)): 0.10
Epoch [17/20], Step[2400/4882], d_loss: 0.0361, g_loss: 6.7542, D(x): 0.98, D(G(z)): 0.01
Epoch [17/20], Step[2500/4882], d_loss: 0.0033, g_loss: 7.7262, D(x): 1.00, D(G(z)): 0.00
Epoch [17/20], Step[2600/4882], d_loss: 0.0627, g_loss: 3.4692, D(x): 0.95, D(G(z)): 0.00
Epoch [17/20], Step[2700/4882], d_loss: 0.0194, g_loss: 6.2217, D(x): 1.00, D(G(z)): 0.02
Epoch [17/20], Step[2800/4882], d_loss: 0.0319, g_loss: 9.3076, D(x): 0.98, D(G(z)): 0.01
Epoch [17/

Epoch [19/20], Step[1400/4882], d_loss: 0.0432, g_loss: 8.2609, D(x): 0.98, D(G(z)): 0.02
Epoch [19/20], Step[1500/4882], d_loss: 0.0033, g_loss: 5.8518, D(x): 1.00, D(G(z)): 0.00
Epoch [19/20], Step[1600/4882], d_loss: 0.0058, g_loss: 9.8204, D(x): 1.00, D(G(z)): 0.00
Epoch [19/20], Step[1700/4882], d_loss: 0.1022, g_loss: 7.1061, D(x): 0.97, D(G(z)): 0.05
Epoch [19/20], Step[1800/4882], d_loss: 0.0078, g_loss: 5.8540, D(x): 1.00, D(G(z)): 0.00
Epoch [19/20], Step[1900/4882], d_loss: 0.1724, g_loss: 5.8564, D(x): 0.91, D(G(z)): 0.01
Epoch [19/20], Step[2000/4882], d_loss: 0.0937, g_loss: 6.5349, D(x): 1.00, D(G(z)): 0.06
Epoch [19/20], Step[2100/4882], d_loss: 0.0424, g_loss: 8.4404, D(x): 0.98, D(G(z)): 0.02
Epoch [19/20], Step[2200/4882], d_loss: 0.0450, g_loss: 8.5811, D(x): 1.00, D(G(z)): 0.04
Epoch [19/20], Step[2300/4882], d_loss: 0.2700, g_loss: 2.3158, D(x): 0.89, D(G(z)): 0.00
Epoch [19/20], Step[2400/4882], d_loss: 0.0158, g_loss: 10.2233, D(x): 0.99, D(G(z)): 0.00
Epoch [19